In [1]:
%load_ext autoreload
%autoreload 2
import logging

logging.getLogger('mat73').setLevel(logging.CRITICAL)
import os
import re
from scipy.io import loadmat
import mat73
logging.getLogger('mat73').setLevel(logging.CRITICAL)

import warnings

from collections import Counter
import pandas as pd
import numpy as np
import mne
from scipy.stats import f_oneway, ttest_ind




import seaborn as sns
import matplotlib.pyplot as plt
# import holoviews as hv
import matplotlib
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42

from utils import *
from connectivity_dynamics import *

In [3]:
DATA_DIR = "/mnt/ernie_main/Price/ephys/data/periconnectivity"
peri_fs = glob.glob(os.path.join(DATA_DIR, "*network*pat*.csv"))

In [ ]:
# fpath = '/mnt/ernie_main/Ghassan/ephys/data/connectivity/Epat23/Epat23_15_FAS_PDC.mat'
# fpath = '/mnt/ernie_main/000_Data/SEEG/SEEG_Periictal/data/Connectivity/seizure_structs_pre10min_ictal_post10min/5sDur_1sStride/Epat23/Epat23_15_FAS_PDC.mat'
# pat_obj = load_mat(fpath)

In [ ]:
# pat_dict = pat_obj['pdc']['seizure']
# pat_dict.keys()

In [4]:
grp_peri_dfs = []
check_list = []
for f in [peri_fs[0]]:
    if "Spat18" in f or "Spat17" in f or "Epat20" in f or 'Epat23' in f:
        continue
    peri_df = pd.read_csv(f)
    try:
        peri_event_df = center_onset(peri_df)
        grp_peri_dfs.append(peri_event_df)
    except IndexError as e:
        print(f) 
        check_list.append(f)
grp_peri_dfs = pd.concat(grp_peri_dfs)

/home/pricewithers/Documents/Research/Ephys/Code/dynamic_ISH/code/connectivity_dynamics.py:632: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  event_df['win_sz_centered'] = center_windows(event_df.window_designations, event_df.period.values,center_designations=center_designations)
/home/pricewithers/Documents/Research/Ephys/Code/dynamic_ISH/code/connectivity_dynamics.py:638: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  event_df['sz_end'] = get_sz_end(event_df)
/home/pricewithers/Documents/Research/Ephys/Cod

In [5]:
peri_df

,period,region,bip,in_pdc,out_pdc,net_pdc,freq_band,window_designations,eventID,patID,sz_type
0,0,soz,H1H2,0.180133,0.070015,0.110119,delta,0.0_0.0_0.0,10,Epat03,"Focal, unknown awareness"
1,0,soz,H2H3,0.194865,0.059261,0.135605,delta,0.0_0.0_0.0,10,Epat03,"Focal, unknown awareness"
2,0,soz,H3H4,0.203958,0.090739,0.113218,delta,0.0_0.0_0.0,10,Epat03,"Focal, unknown awareness"
3,0,soz,A1A2,0.124466,0.088996,0.035470,delta,0.0_0.0_0.0,10,Epat03,"Focal, unknown awareness"
4,0,soz,A2A3,0.123486,0.063086,0.060400,delta,0.0_0.0_0.0,10,Epat03,"Focal, unknown awareness"
...,...,...,...,...,...,...,...,...,...,...,...
4064785,1282,nz,OF4OF5,0.116219,0.078351,0.037868,gamma_H,2.0_2.0_2.0,8,Epat03,FIAS
4064786,1282,nz,OF5OF6,0.153773,0.060794,0.092979,gamma_H,2.0_2.0_2.0,8,Epat03,FIAS
4064787,1282,nz,OF9OF10,0.093915,0.077092,0.016823,gamma_H,2.0_2.0_2.0,8,Epat03,FIAS
4064788,1282,nz,P8P9,0.048678,0.102099,-0.053421,gamma_H,2.0_2.0_2.0,8,Epat03,FIAS


In [ ]:
print(f"Successfully loaded {len(set(grp_peri_dfs.patID))} patients records")
check_list

In [ ]:
# tmp_df = grp_peri_dfs[grp_peri_dfs.period == 0]
# tmp_df  =tmp_df[tmp_df.region == 'soz']
# tmp_df = tmp_df[tmp_df.freq_band == 'delta']
# count_df = tmp_df[['eventID','patID','sz_type']].groupby(['sz_type']).count().reset_index()
# total_sz = count_df.eventID.sum()
# with sns.plotting_context("paper"):
#     with sns.color_palette("cubehelix"):
#         ax = sns.barplot(count_df, x='sz_type', y='eventID', hue="sz_type")
#         _ = plt.xticks(rotation=45)
#         _ = plt.title(f"Count of Seizure Types for {len(set(grp_peri_dfs.patID))} patients, total sz count: {total_sz}")
#         plt.xlabel("Seizure count")
#         plt.ylabel("Seizure Type")
#         plt.savefig("../viz/sz_counts.svg", format='svg', dpi=600, transparent=True)

# Visualizing seizure-centered connectivity dynamics

In [ ]:
grp_peri_dfs = grp_peri_dfs[grp_peri_dfs.freq_band == 'delta']

In [ ]:
grp_peri_dfs

In [ ]:
# Grouping across seizures on a per-patient level
# For every patient, along each region, in each frequency band, for all windows of time per seizure, group and get average values
tmp_df = grp_peri_dfs.dropna()
peri_stats_df = tmp_df[['region','net_pdc','in_pdc','out_pdc','patID','win_sz_st_end','sz_type']].groupby(['patID','region','sz_type','win_sz_st_end']).mean().reset_index()
peri_stats_df

In [ ]:
sz_len = 25
xmin = -60
xmax = sz_len + 60
with sns.plotting_context("paper", font_scale=1.5):
    with sns.color_palette("dark"):
        plot_df = peri_stats_df # [peri_stats_df.patID == 'Epat03']
        plot_df = plot_df[plot_df.win_sz_st_end > xmin]
        plot_df = plot_df[plot_df.win_sz_st_end < xmax]

        g = sns.relplot(
            data=plot_df, 
            row='sz_type',
            row_order=['FAS','FIAS','FBTC'],
            y='net_pdc',
            x='win_sz_st_end',
            hue='region',
            aspect=4,
            kind='line'
        )
        
        for ax, sz_type in zip(g.axes.flatten(), ['FAS','FIAS','FBTC']):
            for i in (0,sz_len):
                ax.axvline(i,ls='--',c='k')
                ax.set_ylabel('Net PDC')
                ax.set_title(sz_type)
        
        g.axes[0][0].text(x=xmin//2, y=0.1, s='Pre-Ictal')
        g.axes[0][0].text(x=((xmax-sz_len)//2)+sz_len, y=0.1, s='Post-Ictal')
        g.axes[-1][0].set_xlabel('Window')
        g.figure.suptitle("Peri-Ictal Net PDC",y=1.03)
        
        plt.savefig("../viz/peri_net_pdc_noNorm_sztypes.pdf", format='pdf', transparent=True)


In [ ]:
sz_len = 25
xmin = -60
xmax = sz_len + 60
with sns.plotting_context("paper", font_scale=1.5):
    with sns.color_palette("dark"):
        plot_df = peri_stats_df # [peri_stats_df.patID == 'Epat03']
        plot_df = plot_df[plot_df.win_sz_st_end > xmin]
        plot_df = plot_df[plot_df.win_sz_st_end < xmax]

        g = sns.relplot(
            data=plot_df, 
            row='sz_type',
            row_order=['FAS','FIAS','FBTC'],
            y='in_pdc',
            x='win_sz_st_end',
            hue='region',
            aspect=4,
            kind='line'
        )
        
        for ax, sz_type in zip(g.axes.flatten(), ['FAS','FIAS','FBTC']):
            for i in (0,sz_len):
                ax.axvline(i,ls='--',c='k')
                ax.set_ylabel('In PDC')
                ax.set_title(sz_type)
        
        g.axes[0][0].text(x=xmin//2, y=0.15, s='Pre-Ictal')
        g.axes[0][0].text(x=((xmax-sz_len)//2)+sz_len, y=0.15, s='Post-Ictal')
        g.axes[-1][0].set_xlabel('Window')
        g.figure.suptitle("Peri-Ictal In PDC",y=1.03)
        
        plt.savefig("../viz/peri_in_pdc_noNorm_sztypes.pdf", format='pdf', transparent=True)


In [ ]:
sz_len = 25
xmin = -60
xmax = sz_len + 60
with sns.plotting_context("paper", font_scale=1.5):
    with sns.color_palette("dark"):
        plot_df = peri_stats_df # [peri_stats_df.patID == 'Epat03']
        plot_df = plot_df[plot_df.win_sz_st_end > xmin]
        plot_df = plot_df[plot_df.win_sz_st_end < xmax]

        g = sns.relplot(
            data=plot_df, 
            row='sz_type',
            row_order=['FAS','FIAS','FBTC'],
            y='out_pdc',
            x='win_sz_st_end',
            hue='region',
            aspect=4,
            kind='line'
        )
        
        for ax, sz_type in zip(g.axes.flatten(), ['FAS','FIAS','FBTC']):
            for i in (0,sz_len):
                ax.axvline(i,ls='--',c='k')
                ax.set_ylabel('Out PDC')
                ax.set_title(sz_type)
        
        g.axes[0][0].text(x=xmin//2, y=0.09, s='Pre-Ictal')
        g.axes[0][0].text(x=((xmax-sz_len)//2)+sz_len, y=0.09, s='Post-Ictal')
        g.axes[-1][0].set_xlabel('Window')
        g.figure.suptitle("Peri-Ictal Out PDC",y=1.03)
        
        plt.savefig("../viz/peri_out_pdc_noNorm_sztypes.pdf", format='pdf', transparent=True)


### Significance Testing
Significance Testing these periods. Want to know if SOZ NIZ and PZ are distinguishable in all 3 periods and also want to know if the connectivity between each period changes significantly.

In [ ]:
# Changes between Periods for Net connectivity
stats_df = window_stats_df[window_stats_df.freq_band =='alpha']
stats_df = stats_df[stats_df.connectivity == 'net_pdc']
windows = list(set(stats_df.win_label))
n = len(windows)
nz_sig_matrix = np.ones((n,n))
pz_sig_matrix = np.ones((n,n))
soz_sig_matrix = np.ones((n,n))
distinguish_reg_matrix = np.ones((n,n))

for i, win_label in enumerate(windows):
    for j, comp_win in enumerate(windows):
        if i == j:
            continue
        win_df = stats_df[stats_df.win_label == win_label]
        comp_df = stats_df[stats_df.win_label == comp_win]

        #Get net connectivity for each region during this window
        soz_win, soz_comp = win_df[win_df.region == 'soz'].strength, comp_df[comp_df.region =='soz'].strength
        pz_win, pz_comp = win_df[win_df.region == 'pz'].strength, comp_df[comp_df.region =='pz'].strength
        nz_win, nz_comp = win_df[win_df.region == 'nz'].strength, comp_df[comp_df.region =='nz'].strength

        nz_sig_matrix[i,j] = ttest_ind(nz_win, nz_comp,permutations=100)[1]
        
        soz_sig_matrix[i,j] = ttest_ind(soz_win, soz_comp,permutations=100)[1]
        pz_sig_matrix[i,j] = ttest_ind(pz_win, pz_comp,permutations=100)[1]
    _, p = f_oneway(soz_win, pz_win,nz_win)
    if p <.05: 
        print(f"In window {win_label} all 3 regions' net connectivity appears to be significantly different\n with p_value:{p}") 
#Setting multiple comparison cut off for significance result
cutoff = .05 / 3*6*6
#highlighting significant entries in the matrix 
soz_sig_matrix[soz_sig_matrix > cutoff] =1
soz_sig_matrix[soz_sig_matrix <= cutoff] =0
#1 - matrix so that significant entries can show up as bright on heatmap (heatmap is of indicators)
soz_win_df = pd.DataFrame(data=1-soz_sig_matrix,columns=windows, index=windows)


nz_sig_matrix[nz_sig_matrix > cutoff] =1
nz_sig_matrix[nz_sig_matrix <= cutoff] =0
nz_win_df = pd.DataFrame(data=1-nz_sig_matrix,columns=windows, index=windows)


pz_sig_matrix[pz_sig_matrix > cutoff] =1
pz_sig_matrix[pz_sig_matrix <= cutoff] =0
pz_win_df = pd.DataFrame(data=1-pz_sig_matrix,columns=windows, index=windows)


In [ ]:
with sns.plotting_context("paper"):
    sns.heatmap(soz_win_df)
    plt.title("SOZ Connectivity Similarity/Diff across periods")

In [ ]:
with sns.plotting_context("paper"):
    sns.heatmap(nz_win_df)
    plt.title("NZ Connectivity Similarity/Diff across periods")

In [ ]:
with sns.plotting_context("paper"):
    sns.heatmap(pz_win_df)
    plt.title("PZ Connectivity Similarity/Diff across periods")

In [ ]:
stats_df = window_stats_df[window_stats_df.freq_band =='alpha']
stats_df = stats_df[stats_df.connectivity == 'net_pdc']
with sns.plotting_context("paper"):
        plot_df = stats_df
        grid = sns.FacetGrid(plot_df, row='region',row_order=['soz', 'nz','pz'],col='win_label', col_order=['interictal','pre_ictal','early_ictal', 'early_post_ictal']) 
        ax = grid.map_dataframe(sns.histplot,x='strength', hue='region',)
        grid.add_legend()
        grid.figure.suptitle(" PDC Histograms",y=1.01)